In [ ]:
import pandas as pd
import json 
import numpy as np

In [ ]:
# the correspodance between categories and amenities
with open('categories-amenities.json') as f:
    categories = json.load(f)

categories

In [ ]:
# original dataframe
# df = pd.read_csv('gdf_oneday_3users.csv')
df = pd.read_csv('nearest_gdf_oneday.csv')
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])
df['label'] = df['label'].apply(int)
df['label'] = df['label'].apply(str)
df['user'] = df['user'].apply(str)
df.head()

In [ ]:
# find duration
df['duration'] = df['end'] - df['start']
df['duration']

In [ ]:
# convert duration in seconds
df['duration'] = df['duration'] / np.timedelta64(1, 's')
df['duration']

In [ ]:
# create new column with empty values
df['category'] = ""

In [ ]:
# find the category
for row, amenity in df['amenity'].iteritems(): 
    for key, values in categories.items():
        if amenity in values:
            df['category'] = key

In [ ]:
df.head()

In [ ]:
new_df = df[["user", "category", "duration"]]
new_df

In [ ]:
# matrix with values
new_df.groupby(['user','category']).sum()